In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
from datetime import datetime

# importing csv files.
crimes_19 = pd.read_csv('input_data/Chicago_Crimes_2019.csv')
crimes_20 = pd.read_csv('input_data/Chicago_Crimes_2020.csv')
crimes_21 = pd.read_csv('input_data/Chicago_Crimes_2021.csv')

# merging crimes data for the last 3 years into a signle dataframe
crimes_data = pd.merge((pd.merge(crimes_19,crimes_20, how="outer")),crimes_21,how="outer")

print(len(crimes_data))


In [ ]:
# converting Date column in dataframe to date format and adding columns for month, year, day, time
crimes_data["Date"] = pd.to_datetime(crimes_data['Date'])
crimes_data["Day"] = crimes_data['Date'].dt.day_name()
crimes_data["Month Day"] =crimes_data['Date'].dt.day
crimes_data["Month"] = crimes_data['Date'].dt.month_name()
crimes_data["Year"] = crimes_data['Date'].dt.year
crimes_data["Time"] =crimes_data['Date'].dt.time
crimes_data["AM_PM"] = pd.to_datetime(crimes_data['Date']).dt.strftime("%p")
print(crimes_data.shape)
crimes_data.dropna(how = 'any', inplace = True)
print(crimes_data.shape)
crimes_data

In [ ]:
# list all the columns titles
crimes_data.columns

In [ ]:
# separate data for each year resulted into arrest 
arrest19_df  = pd.DataFrame(crimes_data[(crimes_data["Year"]==2019) & (crimes_data["Arrest"]== True)])
arrest20_df  = pd.DataFrame(crimes_data[(crimes_data["Year"]==2020) & (crimes_data["Arrest"]== True)])
arrest21_df  = pd.DataFrame(crimes_data[(crimes_data["Year"]==2021) & (crimes_data["Arrest"]== True)])

arrest19_df.head()

In [ ]:
# calculating total arrests in each year
total_arrest_19 = len(arrest19_df["ID"])
total_arrest_20 = len(arrest20_df["ID"])
total_arrest_21 = len(arrest21_df["ID"])
total_crimes_19 = len(crimes_data[crimes_data["Year"]==2019]["ID"])
total_crimes_20 = len(crimes_data[crimes_data["Year"]==2020]["ID"])
total_crimes_21 = len(crimes_data[crimes_data["Year"]==2021]["ID"])
print(f" 019 Arrest = {total_arrest_19}, 020 Arrest = {total_arrest_20}, 021 Arrest = {total_arrest_21}")
print(f" 019 crime = {total_crimes_19}, 020 crime = {total_crimes_20}, 021 crime = {total_crimes_21}")



In [ ]:
# Creating Bar plot for total crimes and arrests

Arrests = [total_arrest_19,total_arrest_20,total_arrest_21]
Total_crimes = [total_crimes_19,total_crimes_20,total_crimes_21]

labels = ["2019", "2020", "2021"]
x = np.arange(len(labels))

width = 0.3  #width of bar plot
fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, Arrests, width, label='Total Arrests')
rects2 = ax.bar(x + width/2, Total_crimes, width, label='Total Crimes')

#Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Crimes')
ax.set_title('Crimes in Chicago')
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()
plt.savefig("Images/chicago_crime_arrest.png")
plt.show()


In [ ]:
#Calculating Percentage arrest in each year
p_arrest2019 = 100*(total_arrest_19/total_crimes_19)
p_arrest2020 = 100*(total_arrest_20/total_crimes_20)
p_arrest2021 = 100*(total_arrest_21/total_crimes_21)

labels = [2019, 2020, 2021]
y = [p_arrest2019,p_arrest2020,p_arrest2021]

x = np.arange(len(labels))
fig, ax = plt.subplots()
width = 0.3
plot1 = ax.bar(x,y, width = 0.4, color = 'r')
ax.set_ylabel("% Arrest")
ax.set_title('Percentage arrest in Chicago')
ax.set_xticks(x, labels)
ax.bar_label(plot1)
plt.savefig("Images/Percentage_arrests.png")

In [ ]:
# Domestic vs non domestic

domestic_crimes_no = len(crimes_data[crimes_data["Domestic"] == True]["ID"])
total_crimes_no = len(crimes_data["ID"])
non_domestic = total_crimes_no - domestic_crimes_no


In [ ]:
#Pi plot to show share of domestic crimes to total crimes

labels = ["Domestic", "Non-domestic"]
sizes = [domestic_crimes_no, non_domestic ]

fig, ax = plt.subplots()
ax.pie(sizes, labels = labels, autopct='%1.1f%%')
plt.savefig("Images/Pie_Domestic_vs_nonDomestic.png")